In [ ]:
!pip install --proxy http://172.20.64.198:8080 -r requirements.txt

In [ ]:
import nest_asyncio
nest_asyncio.apply()
from games.tileman.envs.multi_agent_env import TileServer, ClientPlayerEnv

In [ ]:
proc = TileServer.start_popen_process()

In [ ]:
proc.terminate()

In [ ]:
env = ClientPlayerEnv()
env.reset()

while True:
    res = env.step(0)
    obs, reward, done, truncated, info = res
    if done:
        env.reset()

In [ ]:
import nest_asyncio
nest_asyncio.apply()
import pygame
from stable_baselines3.common.env_util import SubprocVecEnv, make_vec_env, DummyVecEnv
from games.tileman.envs.multi_agent_env import TileServer, ClientPlayerEnv
from stable_baselines3 import PPO, A2C, DQN
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor
import torch.nn as nn
import torch
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor

# vec_env = make_vec_env("tileman-multi-v0", n_envs=4, env_kwargs=dict(vision_range=5))

In [ ]:
from typing import Callable
import gymnasium

def make_env(env_id: str, rank: int, seed: int = 0) -> Callable:
    def _init() -> ClientPlayerEnv:
        env = gymnasium.make(env_id)
        return env

    return _init

env = SubprocVecEnv([make_env("tileman-multi-v0", i) for i in range(6)], start_method="spawn")
# env.reset()

In [ ]:
class CustomCNN(BaseFeaturesExtractor):
    def __init__(self, observation_space, features_dim=256):
        super(CustomCNN, self).__init__(observation_space, features_dim)
        n_input_channels = observation_space.shape[0]
        self.cnn = nn.Sequential(
            nn.Conv2d(n_input_channels, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Flatten(),
        )

        with torch.no_grad():
            n_flatten = self.cnn(torch.as_tensor(observation_space.sample()[None]).float()).shape[1]
        self.linear = nn.Sequential(nn.Linear(n_flatten, features_dim), nn.ReLU())

    def forward(self, observations):
        return self.linear(self.cnn(observations))

policy_kwargs = dict(
    features_extractor_class=CustomCNN,
    features_extractor_kwargs=dict(features_dim=256),
    net_arch=[512, 256, 128, 128, 64],
    normalize_images=False
)

model = PPO("CnnPolicy", env, verbose=0, n_steps=5000, batch_size=100, n_epochs=10, learning_rate=0.0002, policy_kwargs=policy_kwargs)
model.learn(total_timesteps=100_000_000, progress_bar=True, callback=EvalCallback(env, n_eval_episodes=5, eval_freq=10_000, deterministic=True, verbose=1))
obs = env.reset()

In [ ]:
model.save("test")